# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

## Set Global Params

In [3]:
# Random state int:
random_state = 42

# Read the CSV into DataFrame

In [4]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [5]:
df_data_types = df.dtypes.to_frame()
df_data_types

,0
loan_size,float64
interest_rate,float64
homeowner,object
borrower_income,int64
debt_to_income,float64
num_of_accounts,int64
derogatory_marks,int64
total_debt,int64
loan_status,object


In [6]:
# Encode ML unsupported data into binary data
cols_w_unsupported_data_types = list(df_data_types[(df_data_types.iloc[:,0] != 'float64') & (df_data_types.iloc[:,0] != 'int64')].index)
df_binary_encoded = pd.get_dummies(df, columns=cols_w_unsupported_data_types)

# Check df data types after encoding
print(f"Data types : {Counter(list(df_binary_encoded.dtypes.to_frame().iloc[:,0]))}")

Data types : Counter({dtype('uint8'): 5, dtype('int64'): 4, dtype('float64'): 3})


# Split the Data into Training and Testing

In [7]:
# Get 'loan_status' columns to drop
cols_todrop = [str('loan_status_'+var) for var in set(df['loan_status'])]
cols_todrop

['loan_status_low_risk', 'loan_status_high_risk']

In [8]:
# Create our features
X = df_binary_encoded.drop(columns=cols_todrop)

# Create our target
y = df_binary_encoded[cols_todrop[0]]

In [9]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [10]:
# Check the balance of our target values
y.value_counts()

1    75036
0     2500
Name: loan_status_low_risk, dtype: int64

In [11]:
# Train/test split
from sklearn.model_selection import train_test_split

# Create X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_state)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [12]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [13]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [14]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [15]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

In [16]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test_scaled)
print(f"Balanced accuracy score : {round(balanced_accuracy_score(y_test, y_pred),4)}")

Balanced accuracy score : 0.988


In [17]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_test, y_pred), columns=['Pres-0','Pred-1'], index=['0','1'])

,Pres-0,Pred-1
0,581,11
1,102,18690


In [18]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.85      0.98      0.99      0.91      0.99      0.97       592
          1       1.00      0.99      0.98      1.00      0.99      0.98     18792

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [19]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

X_resampled_ros, y_resampled_ros = RandomOverSampler(random_state=random_state).fit_resample(X_train_scaled, 
                                                                                              y_train)

# View the count of target classes with Counter
print(f"Balance of targer values: \n0: {Counter(y_resampled_ros)[0]} | 1: {Counter(y_resampled_ros)[1]}")

Balance of targer values: 
0: 56244 | 1: 56244


In [20]:
# Train the Logistic Regression model using the resampled data
ros_model = LogisticRegression(solver='lbfgs', random_state=random_state)
ros_model.fit(X_resampled_ros, y_resampled_ros)

LogisticRegression(random_state=42)

In [21]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred_ros = ros_model.predict(X_resampled_ros)
print(f"Balanced accuracy score : {round(balanced_accuracy_score(y_resampled_ros, y_pred_ros),4)}")

Balanced accuracy score : 0.994


In [22]:
# Display the confusion matrix
pd.DataFrame(confusion_matrix(y_resampled_ros, y_pred_ros), columns=['Pres-0','Pred-1'], index=['0','1'])

,Pres-0,Pred-1
0,55915,329
1,343,55901


In [23]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_resampled_ros, y_pred_ros))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.99      0.99      0.99      0.99      0.99      0.99     56244
          1       0.99      0.99      0.99      0.99      0.99      0.99     56244

avg / total       0.99      0.99      0.99      0.99      0.99      0.99    112488



### SMOTE Oversampling

In [24]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled_smo, y_resampled_smo = SMOTE(random_state=random_state,
                                         sampling_strategy=1.0).fit_resample(X_train_scaled, 
                                                                                 y_train)

# View the count of target classes with Counter
print(f"Balance of targer values: \n0: {Counter(y_resampled_smo)[0]} | 1: {Counter(y_resampled_smo)[1]}")

Balance of targer values: 
0: 56244 | 1: 56244


In [25]:
# Train the Logistic Regression model using the resampled data
smo_model = LogisticRegression(solver='lbfgs', random_state=random_state)
smo_model.fit(X_resampled_smo, y_resampled_smo)

LogisticRegression(random_state=42)

In [26]:
# Calculated the balanced accuracy score
y_pred_smo = smo_model.predict(X_resampled_smo)
print(f"Balanced accuracy score : {round(balanced_accuracy_score(y_resampled_smo, y_pred_smo),4)}")

Balanced accuracy score : 0.9945


In [27]:
# Display the confusion matrix
pd.DataFrame(confusion_matrix(y_resampled_smo, y_pred_smo), columns=['Pres-0','Pred-1'], index=['0','1'])

,Pres-0,Pred-1
0,55973,271
1,346,55898


In [28]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_resampled_smo, y_pred_smo))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.99      1.00      0.99      0.99      0.99      0.99     56244
          1       1.00      0.99      1.00      0.99      0.99      0.99     56244

avg / total       0.99      0.99      0.99      0.99      0.99      0.99    112488



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [29]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids

X_resampled_cc, y_resampled_cc = ClusterCentroids(random_state=random_state,
                                                  sampling_strategy=1.0).fit_resample(X_train_scaled,
                                                                                      y_train)

# View the count of target classes with Counter
print(f"Balance of targer values: \n0: {Counter(y_resampled_cc)[0]} | 1: {Counter(y_resampled_cc)[1]}")

Balance of targer values: 
0: 1908 | 1: 1908


In [30]:
# Train the Logistic Regression model using the resampled data
cc_model = LogisticRegression(solver='lbfgs', random_state=random_state)
cc_model.fit(X_resampled_cc, y_resampled_cc)

LogisticRegression(random_state=42)

In [31]:
# Calculated the balanced accuracy score
y_pred_cc = cc_model.predict(X_resampled_cc)
print(f"Balanced accuracy score : {round(balanced_accuracy_score(y_resampled_cc, y_pred_cc),4)}")

Balanced accuracy score : 0.9358


In [32]:
# Display the confusion matrix
pd.DataFrame(confusion_matrix(y_resampled_cc, y_pred_cc), columns=['Pres-0','Pred-1'], index=['0','1'])

,Pres-0,Pred-1
0,1891,17
1,228,1680


In [33]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_resampled_cc, y_pred_cc))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.89      0.99      0.88      0.94      0.93      0.88      1908
          1       0.99      0.88      0.99      0.93      0.93      0.86      1908

avg / total       0.94      0.94      0.94      0.94      0.93      0.87      3816



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [34]:
# Resample the data using the SMOTEENN resampler
from imblearn.combine import SMOTEENN

X_resampled_smotn, y_resampled_smotn = SMOTEENN(random_state=random_state, 
                                             sampling_strategy=1.0).fit_resample(X_train_scaled, 
                                                                                 y_train)

# View the count of target classes with Counter
print(f"Balance of targer values: \n0: {Counter(y_resampled_smotn)[0]} | 1: {Counter(y_resampled_smotn)[1]}")

Balance of targer values: 
0: 55603 | 1: 55867


In [35]:
# Train the Logistic Regression model using the resampled data
smotn_model = LogisticRegression(solver='lbfgs', random_state=random_state)
smotn_model.fit(X_resampled_smotn, y_resampled_smotn)

LogisticRegression(random_state=42)

In [36]:
# Calculated the balanced accuracy score
y_pred_smotn = smotn_model.predict(X_resampled_smotn)
print(f"Balanced accuracy score : {round(balanced_accuracy_score(y_resampled_smotn, y_pred_smotn),4)}")

Balanced accuracy score : 0.9983


In [37]:
# Display the confusion matrix
pd.DataFrame(confusion_matrix(y_resampled_smotn, y_pred_smotn), columns=['Pres-0','Pred-1'], index=['0','1'])

,Pres-0,Pred-1
0,55429,174
1,11,55856


In [38]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_resampled_smotn, y_pred_smotn))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      1.00      1.00      1.00      1.00      1.00     55603
          1       1.00      1.00      1.00      1.00      1.00      1.00     55867

avg / total       1.00      1.00      1.00      1.00      1.00      1.00    111470



# Final Questions

1. Which model had the best balanced accuracy score?

   **<code>SMOTEENN()</code> had the best balanced accuracy score of <code>0.9983</code>**

2. Which model had the best recall score?

    **<code>SMOTEENN()</code> had the best recall score of <code>1.00</code>**

3. Which model had the best geometric mean score?

    **<code>SMOTEENN()</code> had the best geometric mean score of <code>1.00</code>**
